In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
import time
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery_storage
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

# Load data from BigQuery

In [2]:
def bq_connector():
    key_path = "C:/Users/HEN1/Projects/Instacart_Market_Basket_Analysis/keys/plucky-mile-327121-255163f80b63.json"
    credentials = service_account.Credentials.from_service_account_file(
        key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
    )

    bqclient = bigquery.Client(credentials=credentials, project=credentials.project_id,)
    bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=credentials)
    return bqclient, bqstorageclient

def bq_full_table_df(bqclient, bqstorageclient, table_name):
    sql_query = f"SELECT * FROM instacart.{table_name}"
    query_job = bqclient.query(sql_query)
    time.sleep(30)
    count =0 
    while query_job.state !='DONE':
        print("NOT DONE")
        if query_job.state =='PENDING':
            print(f"job from {table_name} is pending")
            break
        if query_job.state =='RUNNING':
            print(f"job from {table_name} is running")
            print(query_job.result())
            time.sleep(60)
            query_job.reload()
            time.sleep(10)
            count += 1
            if count>3:
                break
        else:
            print("may meet an error")
            break
    if query_job.state == 'DONE':
        print(f"successfully finished getting data from {table_name} table")
        df = query_job.to_dataframe(bqstorage_client=bqstorageclient, 
                                    progress_bar_type='tqdm_notebook',)
        print("successfully transferred to df")
        time.sleep(3)
    else:
        print("error")

    return df

In [3]:
bqclient, bqstorageclient = bq_connector()

In [4]:
aisles = bq_full_table_df(bqclient, bqstorageclient, 'aisles')
time.sleep(3)

successfully finished getting data from aisles table


Query is running:   0%|          |

Downloading:   0%|          |

successfully transferred to df


In [5]:
departments = bq_full_table_df(bqclient, bqstorageclient, 'departments')
time.sleep(60)

successfully finished getting data from departments table


Query is running:   0%|          |

Downloading:   0%|          |

successfully transferred to df


In [10]:
orders = bq_full_table_df(bqclient, bqstorageclient, 'orders')
time.sleep(60)

NOT DONE
job from orders is running
successfully finished getting data from orders table


Query is running:   0%|          |

Downloading:   0%|          |

successfully transferred to df


In [9]:
products = bq_full_table_df(bqclient, bqstorageclient, 'products')
time.sleep(60)

successfully finished getting data from products table


Query is running:   0%|          |

Downloading:   0%|          |

successfully transferred to df


In [4]:
order_products_prior = bq_full_table_df(bqclient, bqstorageclient, 'order_products_prior')
time.sleep(60)

NOT DONE
job from order_products_prior is running
successfully finished getting data from order_products_prior table


Query is running:   0%|          |

Downloading:   0%|          |

successfully transferred to df


In [5]:
order_products_train = bq_full_table_df(bqclient, bqstorageclient, 'order_products_train')

NOT DONE
job from order_products_train is running
successfully finished getting data from order_products_train table


Query is running:   0%|          |

Downloading:   0%|          |

successfully transferred to df


# Dataset Info

In [12]:
aisles.head()

In [13]:
aisles.info()

In [14]:
departments.head()

In [15]:
departments.info()

In [11]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,3370391,116,prior,1,0,5,NaN
1,2436872,4531,prior,1,0,4,NaN
2,2827892,5553,prior,1,0,5,NaN
3,1513465,8417,prior,1,0,2,NaN
4,713224,13230,prior,1,0,4,NaN


In [12]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 7 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                Int64  
 1   user_id                 Int64  
 2   eval_set                object 
 3   order_number            Int64  
 4   order_dow               Int64  
 5   order_hour_of_day       Int64  
 6   days_since_prior_order  float64
dtypes: Int64(5), float64(1), object(1)
memory usage: 199.0+ MB


In [13]:
orders.eval_set.unique()

array(['prior', 'train', 'test'], dtype=object)

In [15]:
orders.eval_set.value_counts()

prior    3214874
train     131209
test       75000
Name: eval_set, dtype: int64

In [18]:
products.head()

In [19]:
products.info()

In [6]:
order_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,1910419,6366,3,1
1,394918,28221,3,1
2,2989331,26785,3,1
3,3308044,1511,3,1
4,1618145,42888,4,1


In [7]:
order_products_prior.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 4 columns):
 #   Column             Dtype
---  ------             -----
 0   order_id           Int64
 1   product_id         Int64
 2   add_to_cart_order  Int64
 3   reordered          Int64
dtypes: Int64(4)
memory usage: 1.1 GB


In [8]:
order_products_train.head()

,order_id,product_id,add_to_cart_order,reordered
0,98,32463,45,0
1,98,25659,49,0
2,1591,38805,27,0
3,1955,7503,30,0
4,1955,12320,31,0


In [9]:
order_products_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1384617 entries, 0 to 1384616
Data columns (total 4 columns):
 #   Column             Non-Null Count    Dtype
---  ------             --------------    -----
 0   order_id           1384617 non-null  Int64
 1   product_id         1384617 non-null  Int64
 2   add_to_cart_order  1384617 non-null  Int64
 3   reordered          1384617 non-null  Int64
dtypes: Int64(4)
memory usage: 47.5 MB


# Analysis on orders data

In [ ]:
orders.nunique()

In [ ]:
orders.isnull().sum()

In [ ]:
sns.set_palette("GnBu")

### Day of week

In [ ]:
sns.countplot(data=orders, x="order_dow", palette="GnBu")

It is obvious that most orders are on days 0 and 1. Unfortunately there is no info regarding which values represent which day.

### Hour of Day

In [ ]:
sns.countplot(data=orders, x="order_hour_of_day", palette="GnBu")

It is obvious to tell that most orders are between 10:00-16:00.

### Reorder period

In [ ]:
sns.countplot(data=orders, x="days_since_prior_order", palette="GnBu")
plt.xticks(rotation='vertical')
plt.show()

It shows that a lot of people may not place a second order within 1 month. However, if people are willing to reorder in 1 month, they are more likely to place the order after exactly 1 week.


### How many orders for each customer are there?

In [ ]:
customer_last_order = orders.sort_values('order_number').groupby(['user_id']).tail(1)

In [ ]:
plt.figure(figsize=(16, 6))
sns.countplot(data=customer_last_order, x="order_number", palette="GnBu")
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
customer_last_order.order_number.mean()

In [ ]:
customer_last_order.order_number.median()

In [ ]:
customer_last_order.order_number.mode()

The number of orders placed by customers is looked like a heavy-tailed distribution. It can tell that a few orders placed customers have high frequency, while customers placed a lot of orders have low frequency.  
The mean orders placed per customer is around 16.60.  
The median orders placed per customer is around 10.  
The mode orders placed per customer is around 4.


# Analysis on orders and products combined data

In [10]:
products_departments = products.merge(departments, on='department_id', how='left')

In [11]:
products_all = products_departments.merge(aisles, on='aisle_id', how='left')

In [10]:
order_products_prior.head(10)

In [11]:
orders[orders['eval_set']=='test'].head()

In [12]:
orders['eval_set'].unique()

In [13]:
order_products_prior.shape[0]

In [14]:
order_products_prior.order_id.nunique()

In [15]:
order_products_prior.shape[0]/order_products_prior.order_id.nunique()

In [30]:
prior_no_products_by_orders = order_products_prior.order_id.value_counts().rename_axis('order_id').reset_index(name='counts')

In [40]:
plt.figure(figsize=(16, 6))
sns.countplot(data=prior_no_products_by_orders, x="counts", palette="YlGnBu")
plt.xticks(rotation='vertical')
plt.show()

As we can see, there are 32434489 orders and products combination in the prior dataset, and there are 3214874 unique orders, which means there are average 10 products per order. The x-axis of plot represents how many products in the orders, and the y-axis represents how many orders have x-axis number of products. The ditribution has a long-tail as well, which suggests that most orders have 4-7 products. 

## Best Sellers

In [12]:
order_products_prior_combined_products = order_products_prior.merge(products_all, how='left', on='product_id')

In [13]:
order_products_prior_combined_products.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department,aisle
0,606860,9597,10,1,Sweet Kale Chopped Salad,123,4,produce,packaged vegetables fruits
1,1424748,32655,10,1,Organic Large Grade AA Brown Eggs,86,16,dairy eggs,eggs
2,1174241,37646,10,1,Organic Gala Apples,24,4,produce,fresh fruits
3,3123407,3721,11,1,Unsweetened Green Tea,94,7,beverages,tea
4,2349724,18523,11,1,Total 2% All Natural Greek Strained Yogurt wit...,120,16,dairy eggs,yogurt


In [64]:
products_unit_sold = order_products_prior_combined_products.groupby(['product_id'])['product_id'].count().reset_index(name='count').sort_values(['count'], ascending=False).merge(products_all, how='left', on='product_id')

In [81]:
products_unit_sold.shape[0]

In [68]:
products_unit_sold.head(20)

In [86]:
products_unit_sold_gb_aisle = order_products_prior_combined_products.groupby(['aisle_id'])['aisle_id'].count().reset_index(name='count').sort_values(['count'], ascending=False).merge(aisles, how='left', on='aisle_id')

In [88]:
products_unit_sold_gb_aisle.head(20)

In [90]:
products_unit_sold_departments = order_products_prior_combined_products.groupby(['department_id'])['department_id'].count().reset_index(name='count').sort_values(['count'], ascending=False).merge(departments, how='left', on='department_id')

In [91]:
products_unit_sold_departments.head(20)

In [101]:
import seaborn as sns
import matplotlib.pyplot as plt

# Prepare the data from the department dataframe
labels = products_unit_sold_departments['department']
sizes = products_unit_sold_departments['count']
colors = sns.color_palette("YlGnBu")[0:len(labels)]  # Use a pastel color palette

# Create the pie chart
plt.figure(figsize=(12, 8))
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)

# Add a title
plt.title('Distribution of Unit Sold by Departments')

# Display the chart
plt.tight_layout()
plt.show()


From above table, we can see the the top 10 unit sold products are Banana, Bag of Organic Bananas, Organic Strawberries, Organic Baby Spinach, Organic Hass Avocado, Organic Avocado, Large Lemon, Strawberries, Limes, Organic Whole Milk. Most of them are from produce department, fresh fruits aisle. It's pretty common that people love to buy fresh fruits. From above table, we can tell that the most popular products from instacart are daily fundamental fresh foods like fruits and vegitables. 

Here are the top five aisles by count:

1. Fresh Fruits (Aisle ID: 24) - Count: 3,642,188
2. Fresh Vegetables (Aisle ID: 83) - Count: 3,418,021
3. Packaged Vegetables Fruits (Aisle ID: 123) - Count: 1,765,313
4. Yogurt (Aisle ID: 120) - Count: 1,452,343
5. Packaged Cheese (Aisle ID: 21) - Count: 979,763

Here are the top five departments by count:

1. Produce (Department ID: 4) - Count: 9,479,291
2. Dairy Eggs (Department ID: 16) - Count: 5,414,016
3. Snacks (Department ID: 19) - Count: 2,887,550
4. Beverages (Department ID: 7) - Count: 2,690,129
5. Frozen (Department ID: 1) - Count: 2,236,432

There are some insights we can provide from above information. 

1. The top aisles and departments with the highest counts indicate the most popular categories among customers. Allocate resources and marketing efforts to promote and showcase these popular aisles and departments to drive sales. For example, highlight fresh fruits, fresh vegetables, dairy eggs, snacks, and beverages, as these are highly sought-after categories.

2. Cross-Promote Related Aisles and Departments: Identify complementary aisles and departments that can be cross-promoted to encourage customers to explore and purchase items from multiple categories. For instance, promote packaged vegetables with fresh fruits, or pair yogurt with granola bars or energy bars. This can increase basket size and drive overall sales.

3. Improve Online Visibility and Navigation: Optimize the e-commerce platform to enhance the visibility and accessibility of popular aisles and departments. Ensure that customers can easily find and navigate through the relevant categories. Implement effective search filters, intuitive navigation menus, and personalized recommendations to enhance the user experience and drive sales.


## Most Reordered Products

In [103]:
order_products_prior_combined_products.info()

In [109]:
# Group the dataframe by product_id, calculate proportion of reordered being 1, reset index, rename column, and sort in descending order
proportion_reordered = order_products_prior_combined_products.groupby('product_id').agg(occurrence=('product_id', 'count'), reordered_rate=('reordered', 'mean')).reset_index().sort_values('reordered_rate', ascending=False).merge(products_all, how='left', on='product_id')


In [115]:
# Display the result
print(proportion_reordered.head(20))


1. Reordered Rate: The products in the top 20 list exhibit high reordered rates, ranging from 0.88 to 0.941176. This suggests that customers are highly likely to reorder these products, indicating strong customer satisfaction and preference for these items.

2. Department and Aisle Analysis: The top 20 products are spread across various departments and aisles, including produce, personal care, beverages, snacks, babies, pantry, alcohol, and others. This indicates that customers have diverse preferences when it comes to repurchasing items, spanning multiple categories and departments.

3. Cross-Selling Opportunities: The analysis reveals potential cross-selling opportunities within specific departments. For example, customers purchasing items from the snacks department (e.g., candy, chocolate) are likely to reorder related snack items. Instacart can optimize product placements, suggest related products, and create bundles to encourage customers to explore complementary items.

4. Customer Preferences: The top 20 products provide valuable insights into customer preferences and buying patterns. Instacart can use this information to optimize inventory management, ensure the availability of popular products, and identify emerging trends or shifts in customer preferences.



## Which item do people put into the cart first?

In [18]:
first_product = order_products_prior_combined_products[order_products_prior_combined_products['add_to_cart_order']==1].copy()

In [14]:
order_products_prior_combined_products.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department,aisle
0,606860,9597,10,1,Sweet Kale Chopped Salad,123,4,produce,packaged vegetables fruits
1,1424748,32655,10,1,Organic Large Grade AA Brown Eggs,86,16,dairy eggs,eggs
2,1174241,37646,10,1,Organic Gala Apples,24,4,produce,fresh fruits
3,3123407,3721,11,1,Unsweetened Green Tea,94,7,beverages,tea
4,2349724,18523,11,1,Total 2% All Natural Greek Strained Yogurt wit...,120,16,dairy eggs,yogurt


In [21]:
cart_first_product = order_products_prior_combined_products.groupby('product_id').agg({
    'add_to_cart_order': lambda x: sum(x == 1),
    'product_id': 'count'
}).rename(columns={'add_to_cart_order': 'order_first_count', 'product_id': 'product_count'}).sort_values(
    ['order_first_count'], ascending=False)

cart_first_product['percentage'] = cart_first_product['order_first_count'] / cart_first_product['product_count']
cart_first_product = cart_first_product.merge(products, how='left', on='product_id')



In [22]:
cart_first_product.head(10)

,product_id,order_first_count,product_count,percentage,product_name,aisle_id,department_id
0,24852,110916,472565,0.234711,Banana,24,4
1,13176,78988,379450,0.208164,Bag of Organic Bananas,24,4
2,27845,30927,137905,0.224263,Organic Whole Milk,84,16
3,21137,27975,264683,0.105692,Organic Strawberries,24,4
4,47209,24116,213584,0.112911,Organic Hass Avocado,24,4
5,21903,23543,241921,0.097317,Organic Baby Spinach,123,4
6,47766,22398,176815,0.126675,Organic Avocado,24,4
7,19660,16822,56087,0.299927,Spring Water,115,7
8,16797,16366,142951,0.114487,Strawberries,24,4
9,27966,14393,137057,0.105015,Organic Raspberries,123,4


In [26]:
cart_first_product[cart_first_product['order_first_count']>cart_first_product['order_first_count'].mean()].sort_values(['percentage'], ascending=False).head(10)

,product_id,order_first_count,product_count,percentage,product_name,aisle_id,department_id
5911,47765,70,111,0.630631,Alimentum Infant Formula,92,18
5702,6209,74,155,0.477419,TAB,77,7
3292,10031,146,311,0.469453,Homestyle Orange Juice,31,7
436,35470,1051,2244,0.46836,Water Mineral,115,7
3397,40462,142,307,0.462541,Cleanse,31,7
4922,20814,90,202,0.445545,Sensitive with Iron Infant Formula,92,18
4074,34568,114,259,0.440154,Bourbon Kentucky Frontier Whiskey,124,5
725,11885,684,1556,0.439589,"Sparkling Water, Bottles",115,7
753,45004,662,1531,0.432397,White Multifold Towels,54,17
3924,31380,119,277,0.429603,Omeprazole Acid Reducer Tablets,70,11


The 'order_first_count' column represents the count of occurrences where a specific product was added as the first item in the cart. By analyzing this count, you can identify which products are most frequently chosen as the initial item by customers. These products are pretty popular from above tables. By comparing the 'product_count' and 'order_first_count' columns, you can identify products that have a high overall demand but a relatively low count of being added as the first item. This discrepancy may indicate an opportunity to promote these products more effectively or improve their visibility to capture the first choice of customers. Then we have filter table by order_first_count is greater than the mean of the this column, which can filter those products with low first add to cart frequency. We can see the top 10 percentage rate of these products. We can get some insights by these 2 tables:
1. High first item count and high percentage: Products that appear in the top positions of both dataframes have a high count of being added as the first item (order_first_count) and a high percentage of occurrences as the first item (percentage). These products are consistently popular and preferred by customers as their initial choice. They represent strong contenders for featured placement, promotional campaigns, or upselling strategies.

2. High first item count and lower percentage: Products that have a high count of being added as the first item but a lower percentage in the second dataframe may indicate a high overall demand for the product, but it is not always the first choice for customers. These products could present opportunities for improving their visibility or positioning to capture more first item selections.

3. Lower first item count and high percentage: Products that have a lower count of being added as the first item but a high percentage in the second dataframe suggest that although these products are not frequently added as the first item, they have a strong association with other items. Consider exploring cross-selling opportunities or creating bundles/packages featuring these products to leverage their popularity and increase their first item selection rate.

# Visualize the products categories

In [32]:
order_products_prior_combined_products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32434489 entries, 0 to 32434488
Data columns (total 9 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   order_id           Int64 
 1   product_id         Int64 
 2   add_to_cart_order  Int64 
 3   reordered          Int64 
 4   product_name       object
 5   aisle_id           Int64 
 6   department_id      Int64 
 7   department         object
 8   aisle              object
dtypes: Int64(6), object(3)
memory usage: 2.6+ GB


In [39]:
aisle_count = order_products_prior_combined_products.groupby(['department', 'aisle']).size().reset_index(name='count')

# Plot the treemap using plotly express
fig = px.treemap(aisle_count, path=['department', 'aisle'], values='count')

# Update layout for better visibility
fig.update_layout(
    title='Aisles Organized Within Departments',
    width=1200,
    height=800,
    margin=dict(t=40, l=0, r=0, b=40),
)

fig.show()

In [48]:
# Join order details to prior orders
dt_orders_prior = pd.merge(orders[orders['eval_set'] == 'prior'], order_products_prior, on='order_id')


# Summarise orders by product and hour
dt_product_hour = dt_orders_prior.groupby(['product_id', 'order_hour_of_day']).size().reset_index(name='count')
dt_product_sum = dt_orders_prior.groupby('product_id').size().reset_index(name='total')
dt_product_hour = pd.merge(dt_product_hour, dt_product_sum, on='product_id')
dt_product_hour['pcnt_orders'] = dt_product_hour['count'] / dt_product_hour['total']

# Create summary of top 25 morning and afternoon products
morning_products = dt_product_hour[(dt_product_hour['total'] > 3000) & (dt_product_hour['order_hour_of_day'].isin([9, 10, 11]))].nlargest(25, 'pcnt_orders')
afternoon_products = dt_product_hour[(dt_product_hour['total'] > 3000) & (dt_product_hour['order_hour_of_day'].isin([15, 16, 17]))].nlargest(25, 'pcnt_orders')
dt_product_hour_subset = pd.concat([morning_products, afternoon_products], ignore_index=True)

# Merge on product description
dt_product_hour_subset = pd.merge(dt_product_hour_subset[['product_id', 'pcnt_orders', 'order_hour_of_day']], products[['product_id', 'product_name']], on='product_id', how='left')

# Extract summary of all data for those products
dt_product_hour_subset2 = pd.merge(dt_product_hour_subset['product_id'], dt_product_hour, on='product_id', how='left')

# Create a plot of the data
lst_products = dt_product_hour_subset['product_id'].unique()

In [95]:
# Set up a plotly container
fig = go.Figure()

# Loop through product list adding line to chart
for product_id in lst_products:
    trace_colour = "rgba(7, 138, 31, 0.4)" if np.any(dt_product_hour_subset[dt_product_hour_subset['product_id'] == product_id]['order_hour_of_day'] < 12) else "rgba(255, 0, 0, 0.4)"
    x = dt_product_hour_subset2[dt_product_hour_subset2['product_id'] == product_id]['order_hour_of_day']
    y = 100 * dt_product_hour_subset2[dt_product_hour_subset2['product_id'] == product_id]['pcnt_orders']
    fig.add_trace(go.Scatter(x=x, y=y, mode='lines', line=dict(color=trace_colour)))

# Add vertical line at midday
fig.add_shape(type="line", x0=12, x1=12, y0=0, y1=100, line=dict(color="rgb(211, 211, 211)", dash='dash'))

# Set up annotations
annotate_am = dict(
    xref='x',
    yref='y',
    x=-1,
    y=20,
    xanchor='left',
    yanchor='top',
    text='<br>'.join(dt_product_hour_subset[dt_product_hour_subset['order_hour_of_day'] < 12]['product_name']),
    font=dict(family='Arial', size=10, color="rgb(7, 138, 31)"),
    align='left',
    showarrow=False
)


annotate_pm = dict(
    xref='x',
    yref='y',
    x=24,
    y=20,
    xanchor='right',
    yanchor='top',
    text='<br>'.join(dt_product_hour_subset[dt_product_hour_subset['order_hour_of_day'] >= 12]['product_name']),
    font=dict(family='Arial', size=10, color="rgb(255, 0, 0)"),
    align='right',
    showarrow=False
)

# Add titles and layout
fig.update_layout(
    xaxis=dict(
        range=[-1, 24],
        title="Hour of Day Ordered",
        title_font=dict(family="Arial, Bold", size=18, color="#000000"),
        tickfont=dict(family="Arial", size=12, color="#000000"),
        tickmode="auto",
        nticks=13,
        ticklen=7,
        tickwidth=2,
        ticks="outside",
        showline=False,
        zeroline=False,
        showgrid=False
    ),
    yaxis=dict(
        range=[-1, 20],
        title="Percent of Orders by Product",
        title_font=dict(family="Arial, Bold", size=18, color="#000000"),
        tickfont=dict(family="Arial", size=12, color="#000000"),
        tickmode="auto",
        nticks=4,
        ticks="outside",
        ticklen=7,
        tickwidth=2,
        showline=False,
        zeroline=False,
        showticklabels=True,
        showgrid=False
    ),
    showlegend=False,
    margin=dict(l=70, b=60, r=100),
)
fig.update_layout(annotations=[annotate_am, annotate_pm])


# Display the plot
fig.show()